# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes (helps with RAM usage).
-   Impute missing feature values with forward, backward and then 0 filling.
-   Split the dataset into multiple cross validation folds.
-   Standardize feature values.
-   Pad/Truncate the sequences to the same length.  

> Note:  
> - Demographics data set will be ignored for now.  

## Setup

### Imports

In [14]:
import os
import json
import warnings
from os.path import join
from tqdm.notebook import tqdm
from itertools import repeat, starmap

import numpy as np
import pandas as pd
from numpy import ndarray
import plotly.express as px
from pandas import DataFrame as DF
from scipy.spatial.transform import Rotation
from kagglehub import whoami, competition_download, dataset_upload

from config import *

### Supress performance warngings

In [15]:
warnings.filterwarnings(
    "ignore",
    message=(
        "DataFrame is highly fragmented.  This is usually the result of "
        "calling `frame.insert` many times.*"
    ),
    category=pd.errors.PerformanceWarning,
)

### Define function to get the feature columns
Feature columns change over time so it's better to have a function to get them than manually update a variable every time we add/remove features.

In [16]:
def get_feature_cols(df:DF) -> list[str]:
    return list(set(df.columns) - set(META_DATA_COLUMNS) - set(df["gesture"].unique().tolist()))

## Data preprocessing

### Load dataset
Requires to be logged in if this notebook is not running on kaggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [ ]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

### Impute missing data
Perform forward, backward and then filling of all NaN sequences.

In [ ]:
# Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
# So we replace them by NaN and then perform imputing.
tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

def imputed_features(df:DF) -> DF:
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
    fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

    df[get_feature_cols(df)] = (
        df
        .loc[:, get_feature_cols(df)]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(tof_vals_to_nan, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(fillna_val_per_col)
    )
    return df

df = imputed_features(df)

### Norm quaternions
This allows us to parse quaternions with `scipy.spatial.transform.Rotation`.

In [ ]:
QUATERNION_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
def norm_quat_rotations(df:DF) -> DF:
    df[QUATERNION_COLS] /= np.linalg.norm(df[QUATERNION_COLS], axis=1, keepdims=True)
    return df

### Add gravity free acceleration features

In [ ]:
GRAVITY_WORLD = np.array([0, 0, 9.81], "float32")
ACCELRATION_COLS = ["acc_x", "acc_y", "acc_z"]
GRAVITY_FREE_ACC_COLS = ["gravity_free_" + col for col in ACCELRATION_COLS]
def add_gravity_free_acc_cols(df:DF) -> DF:
    # Vectorized version of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Dataset `remove_gravity_from_acc`
    rotations:Rotation = Rotation.from_quat(df[QUATERNION_COLS])
    gravity_sensor_frame = rotations.apply(GRAVITY_WORLD, inverse=True).astype("float32")
    df[GRAVITY_FREE_ACC_COLS] = df[ACCELRATION_COLS] - gravity_sensor_frame
    return df

df = add_gravity_free_acc_cols(df)
df[get_feature_cols(df)].dtypes

tof_3_v43    float32
tof_3_v46    float32
tof_4_v25    float32
tof_4_v29    float32
tof_1_v56    float32
              ...   
tof_1_v40    float32
tof_3_v49    float32
tof_3_v23    float32
tof_3_v26    float32
tof_5_v1     float32
Length: 335, dtype: object

### Euler angles from quaternions

In [ ]:
EULER_ANGLES_COLS = ["euler_x", "euler_y", "euler_z"]
def rot_euler_angles(df:DF) -> ndarray:
    df[EULER_ANGLES_COLS] = (
        Rotation
        .from_quat(df[QUATERNION_COLS])
        .as_euler("xyz")
        .squeeze()
    )
    return df

df = rot_euler_angles(df)

### One hot encode target values.

In [ ]:
one_hot_target = pd.get_dummies(df["gesture"])
df[one_hot_target.columns] = one_hot_target
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Neck - scratch,Text on phone,Wave hello,Write name in air,Write name on leg,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,False,False,False,False,False
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,False,False,False,False,False
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,False,False,False,False,False
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,False,False,False,False,False
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,True,False,False,False,False,False
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,True,False,False,False,False,False
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,True,False,False,False,False,False
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,True,False,False,False,False,False


### ToF data aggregation.
Time of Flight columns take most of the data, let's reduce their size by aggregating by mean for each Time of Flight sensor.

In [ ]:
def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in range(1, 6):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = (
            df
            # Need to use a dict otherwise the name of the col will be "tof_preffix" instead of the value it contains
            .assign(**{tof_name:df[tof_cols].mean(axis="columns")})
            .drop(columns=tof_cols)
        )
    return df

df = agg_tof_cols_per_sensor(df)
df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off,tof_1,tof_2,tof_3,tof_4,tof_5
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,139.250000,117.109375,91.687500,123.359375,135.343750
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,139.796875,119.671875,97.921875,124.406250,137.000000
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,142.375000,128.359375,116.953125,125.687500,140.234375
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,154.109375,142.093750,144.515625,149.078125,142.609375
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,177.953125,149.453125,161.828125,163.765625,151.265625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,False,68.562500,67.750000,144.437500,74.062500,52.843750
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,False,70.234375,66.656250,144.000000,70.406250,54.531250
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,False,66.671875,62.906250,136.906250,70.109375,57.468750
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,False,67.218750,64.500000,140.531250,75.609375,54.937500


### Add derivative w.r.t time features

In [ ]:
def add_diff_features(df:DF) -> DF:
    df[[col + "_diff" for col in get_feature_cols(df)]] = (
        df
        .groupby("sequence_id", observed=True)
        [get_feature_cols(df)]
        .diff()
        .values
    )
    return df

df = add_diff_features(df)

In [ ]:
get_feature_cols(df)

AttributeError: 'NoneType' object has no attribute 'columns'

### Split into folds

In [ ]:
def split_dataset(df:DF, by="subject") -> tuple[DF, DF]:
    unique_sequences = df[by].unique()
    validation_sequences = pd.Series(unique_sequences).sample(
        frac=VALIDATION_FRACTION, replace=False
    )

    validation_set = df[df[by].isin(validation_sequences)].copy()
    train_set = df[~df[by].isin(validation_sequences)].copy()

    return train_set, validation_set

folds = list(map(split_dataset, repeat(df, N_FOLDS)))

TypeError: 'NoneType' object is not subscriptable

### Std norm
Standard scale the feature cols (should probably do something different for IMU cols).  
<!-- *Deprecated, std norm is now performed at dataset creation to avoid target leakage.*   -->

In [ ]:
def std_norm_dataset(train:DF, val:DF) -> tuple[DF, DF]:
    means = train[get_feature_cols(df)].mean().astype("float32")
    stds = train[get_feature_cols(df)cols(df)].std().astype("float32")
    train.loc[:, get_feature_cols(df)cols(df)] = (train[get_feature_cols(df)] - means) / stds
    val.loc[:, get_feature_cols(df)] = (val[get_feature_cols(df)] - means) / stds
    return train, val

folds = list(starmap(std_norm_dataset, folds))

Normalize full dataset.

In [ ]:
# Retain full dataset meta data for inference
full_dataset_meta_data = {
    "mean": df[get_feature_cols(df)].mean().astype("float32").to_dict(),
    "std": df[get_feature_cols(df)].std().astype("float32").to_dict(),
}
df.loc[:, get_feature_cols(df)] = (df[get_feature_cols(df)] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']

Verify the mean and std of the full dataset.

In [ ]:
df[get_feature_cols(df)].agg(["mean", "std"])

,gravity_free_acc_y,tof_5,thm_3,thm_1,rot_z,thm_5,tof_2,acc_y,tof_4,thm_4,...,tof_1,rot_x,tof_3,gravity_free_acc_z,euler_x,rot_w,gravity_free_acc_x,euler_y,acc_x,acc_z
mean,2.548818e-08,-5.993022e-08,1.823665e-07,3.772558e-08,1.855411e-08,-3.565034e-09,1.687596e-08,1.186949e-08,1.404808e-08,-1.427808e-07,...,1.908273e-07,-2.663767e-08,2.663846e-09,-8.419120e-09,1.581881e-17,-6.315844e-08,1.358779e-09,1.581881e-18,-1.749467e-08,-2.971537e-09
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Let's compare the train to validation mean/std skews.

In [ ]:
pd.concat({
    "train": folds[0][0][get_feature_cols(df)].agg(["mean", "std"]),
    "validation": folds[0][1][get_feature_cols(df)].agg(["mean", "std"]),
})

gravity_free_acc_y         tof_5         thm_3         thm_1  \
train      mean        2.122192e-09  8.064328e-08 -5.485865e-07  3.374284e-07   
           std         1.000000e+00  1.000000e+00  9.999999e-01  1.000000e+00   
validation mean       -4.839994e-02  7.867061e-02  1.439791e-01  1.279001e-01   
           std         9.832454e-01  8.796210e-01  4.859858e-01  5.892192e-01   

                        rot_z         thm_5         tof_2         acc_y  \
train      mean  1.008041e-08  9.125424e-08  1.119456e-07  2.055873e-08   
           std   1.000000e+00  1.000000e+00  9.999999e-01  9.999999e-01   
validation mean  6.550998e-03  2.419055e-01 -1.180771e-02  1.990715e-02   
           std   1.056812e+00  3.386279e-01  9.516543e-01  1.009363e+00   

                        tof_4         thm_4  ...         tof_1         rot_x  \
train      mean -1.671226e-08  2.440520e-08  ...  2.551935e-07  1.697753e-08   
           std   1.000000e+00  1.000000e+00  ...  1.000000e+00  9.999999e-01   
validation mean -1.203539e-02  1.086784e-01  ... -1.693127e-02  1.458404e-01   
           std   9.791272e-01  6.438745e-01  ...  9.645705e-01  1.064583e+00   

                        tof_3  gravity_free_acc_z       euler_x         rot_w  \
train      mean  1.830390e-08        2.918013e-09  2.385624e-10 -3.448561e-08   
           std   1.000000e+00        1.000000e+00  1.000000e+00  1.000000e+00   
validation mean -1.934037e-02        1.285326e-02 -2.707513e-03 -1.051750e-01   
           std   9.729294e-01        9.447199e-01  9.868742e-01  9.654092e-01   

                 gravity_free_acc_x       euler_y         acc_x         acc_z  
train      mean        1.671226e-08 -1.046622e-08  2.732322e-08 -9.019314e-09  
           std         9.999999e-01  1.000000e+00  9.999999e-01  1.000000e+00  
validation mean       -2.170409e-02 -1.806605e-02 -2.172979e-02  3.759031e-02  
           std         9.949074e-01  9.939340e-01  9.884086e-01  1.001982e+00  

[4 rows x 23 columns]

### Normalize sequences lengths.  
And turn the Dataframes into ndarrays.

#### Visualize histogram of sequences lengths.

Entire dataset sequences lengths.

In [ ]:
px.histogram(
    (
        df
        .groupby("sequence_id", observed=True)
        .size()
    ),
    title="Sequence length frequency",
)

Second(to avoid always look at the first one) Train/validation split sequences lengths comparaison.

In [ ]:
def get_set_sequences_lengths(set:DF, name:str) -> DF:
    return (
        set
        .groupby("sequence_id", observed=True)
        .size()
        .reset_index(name="length")
        .assign(set=name)
    )

full_se_lengths = pd.concat((
    get_set_sequences_lengths(folds[2][0], "Train"),
    get_set_sequences_lengths(folds[2][1], "Validation"),
))

fig = px.histogram(
    full_se_lengths,
    x="length",
    color="set",
    barmode="overlay",  # or 'group' if you want side-by-side bars
    nbins=50,           # adjust bin size if needed
    title="Sequence Length Distribution: Train vs Validation"
)

fig.update_traces(opacity=0.8)  # better visibility with overlay
fig.show()


In [ ]:
for train, val in folds:
    print("train normed sequence len:", int(train.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print("validation normed sequence len:", int(val.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print()

train normed sequence len: 129
validation normed sequence len: 124

train normed sequence len: 124
validation normed sequence len: 141

train normed sequence len: 124
validation normed sequence len: 136

train normed sequence len: 127
validation normed sequence len: 124

train normed sequence len: 126
validation normed sequence len: 131



#### Sequence length norm implementation

In [ ]:
gesture_cols = df["gesture"].unique()

def length_normed_sequence_feat_arr(sequence: DF, normed_sequence_len: int) -> ndarray:
    features = (
        sequence
        .loc[:, get_feature_cols(df)]
        .values
    )
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def df_to_ndarrays(df:DF, normed_sequence_len:int) -> tuple[np.ndarray, np.ndarray]:
    sequence_it = df.groupby("sequence_id", observed=True, as_index=False)
    x = np.empty(
        shape=(len(sequence_it), normed_sequence_len, len(get_feature_cols(df))),
        dtype="float32"
    )
    y = np.empty(
        shape=(len(sequence_it), df["gesture"].nunique()),
        dtype="float32"
    )
    for sequence_idx, (_, sequence) in tqdm(enumerate(sequence_it), total=len(sequence_it)):
        normed_seq_feat_arr = length_normed_sequence_feat_arr(sequence, normed_sequence_len)
        x[sequence_idx] = normed_seq_feat_arr
        # Take the first value as they are(or at least should be) all the same in a single sequence
        y[sequence_idx] = sequence[gesture_cols].iloc[0].values

    return x, y

def get_normed_seq_len(dataset:DF) -> int:
    return int(
        dataset
        .groupby("sequence_id", observed=True)
        .size()
        .quantile(SEQUENCE_NORMED_LEN_QUANTILE)
    )

def fold_dfs_to_ndarrays(train:DF, validation:DF) -> tuple[ndarray, ndarray, ndarray, ndarray]:
    """
    Returns:
        (train X, train Y, validation X, validation Y)
    """
    normed_sequence_len = get_normed_seq_len(train)
    return (
        *df_to_ndarrays(train, normed_sequence_len),
        *df_to_ndarrays(validation, normed_sequence_len),
    )

folds_arrs = list(starmap(fold_dfs_to_ndarrays, folds))

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

In [ ]:
full_dataset_sequence_length_norm = get_normed_seq_len(df)
full_x, full_y = df_to_ndarrays(df, full_dataset_sequence_length_norm)

  0%|          | 0/8151 [00:00<?, ?it/s]

## Create dataset

In [ ]:
# Clean dataset directory if it already exists
! rm -rf preprocessed_dataset
# Create dataset direcory
! mkdir preprocessed_dataset
# Save folds
for fold_i, (train_x, train_y, val_x, val_y) in enumerate(folds_arrs):
    fold_dir_path = join("preprocessed_dataset", f"fold_{fold_i}")
    os.makedirs(fold_dir_path)
    # save features (X)
    np.save(join(fold_dir_path, "train_X.npy"), train_x, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_X.npy"), val_x, allow_pickle=False)
    # Save targets (Y)
    np.save(join(fold_dir_path, "train_Y.npy"), train_y, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_Y.npy"), val_y, allow_pickle=False)
# Save full dataset
full_dataset_dir_path = "preprocessed_dataset/full_dataset"
os.makedirs(full_dataset_dir_path)
np.save(join(full_dataset_dir_path, "X.npy"), full_x, allow_pickle=False)
np.save(join(full_dataset_dir_path, "Y.npy"), full_y, allow_pickle=False)
# Save dataset meta data
full_dataset_meta_data["target_names"] = one_hot_target.columns.to_list()
full_dataset_meta_data["pad_seq_len"] = full_dataset_sequence_length_norm
full_dataset_meta_data["get_feature_cols(df)"] = get_feature_cols(df)

with open("preprocessed_dataset/full_dataset_meta_data.json", "w") as fp:
    json.dump(full_dataset_meta_data, fp, indent=4)

In [ ]:
df[get_feature_cols(df)]

,gravity_free_acc_y,tof_5,thm_3,thm_1,rot_z,thm_5,tof_2,acc_y,tof_4,thm_4,...,tof_1,rot_x,tof_3,gravity_free_acc_z,euler_x,rot_w,gravity_free_acc_x,euler_y,acc_x,acc_z
0,0.378283,0.404678,0.633442,0.502318,-1.238540,0.477938,0.148727,0.884130,-0.016778,0.367246,...,0.339571,-0.508525,-0.454938,-0.191143,0.191240,-0.997151,0.921929,0.883372,0.872408,0.625816
1,0.441917,0.433771,0.681437,0.593105,-1.267920,0.492298,0.199531,0.884130,0.002596,0.386687,...,0.349948,-0.476437,-0.337148,-0.239702,0.150843,-0.957737,0.935809,0.860138,0.918353,0.588012
2,0.663317,0.490583,0.741405,0.821468,-1.349747,0.510858,0.371769,0.723320,0.026308,0.423273,...,0.398864,-0.334147,0.022423,0.018062,-0.047236,-0.630264,0.779645,0.754947,0.706192,0.964766
3,0.700636,0.532300,0.584725,0.868207,-1.390175,0.341522,0.644067,0.347835,0.459197,0.484648,...,0.621508,-0.312448,0.543179,0.236127,-0.231340,-0.290093,0.839486,0.714206,0.858218,1.134561
4,0.487647,0.684347,0.081696,0.587718,-1.447599,0.334854,0.789974,-0.302433,0.731018,0.592139,...,1.073910,-0.214344,0.870276,0.639598,-0.342103,-0.134027,0.600054,0.418963,0.679165,1.655481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,-1.541394,-1.044439,0.292463,0.590561,0.070326,0.656701,-0.829872,-0.444509,-0.929112,0.854270,...,-1.001627,-1.601608,0.541703,-0.166781,1.457859,-1.117506,0.345242,0.287935,0.322408,-1.309212
574941,-1.547135,-1.014798,0.314339,0.599804,0.045559,0.618153,-0.851557,-0.490566,-0.996778,0.870208,...,-0.969906,-1.607920,0.533437,-0.249065,1.471826,-1.100313,0.390242,0.326326,0.369030,-1.434796
574942,-1.418637,-0.963200,0.299698,0.620099,0.041674,0.618153,-0.925905,-0.314143,-1.002272,0.848956,...,-1.037499,-1.605422,0.399410,-0.024379,1.470549,-1.089203,0.333282,0.335326,0.249436,-1.138775
574943,-1.473273,-1.007662,0.330572,0.695652,0.017150,0.561656,-0.894307,-0.429677,-0.900484,0.864896,...,-1.027123,-1.610813,0.467900,-0.223986,1.484620,-1.070422,0.433050,0.374099,0.402138,-1.414933


## Dataset upload
Optionally upload the dataset to kaggle.

In [ ]:
if input("Do you want to upload the  dataset to kaggle?[yes/no]").lower() == "yes":
    # Updaload the dataset
    dataset_upload(
        join(whoami()["username"], "prepocessed-cmi-2025"),
        "preprocessed_dataset",
        version_notes="Preprocessed Child Mind Institue 2025 competition dataset."
    )
else:
    print("Dataset has not been uploaded.")

Dataset has not been uploaded.
